In [ ]:
##old function for formatting original network data

# def draw_weighted_colored_graph(u, edge_color_property_name='edge_color'):
#     """
#     Prepares edge color and pen width based on 'count' and 'pctSingleBid' properties.
#     Designed to be used within gt.graph_draw.
#     :param u: The graph for which the drawing parameters are being prepared.
#     :param edge_color_property_name: The name to be assigned to the new edge color property.
#     :return: A dictionary with the edge_color and edge_pen_width properties.
#     """
#     # Retrieve the 'count' and 'pctSingleBid' edge properties
#     edge_count = u.ep["count"]
#     edge_pctSingleBid = u.ep["pctSingleBid"]

#     # Create a new edge property for the matplotlib color mapping
#     edge_color = u.new_edge_property('vector<double>')
#     u.ep[edge_color_property_name] = edge_color

#     # Create a color map and normalization instance
#     cmap = plt.get_cmap('coolwarm')
#     norm = colors.Normalize(vmin=0, vmax=1)  # 'pctSingleBid' is always between 0 and 1

#     # Assign colors to edges based on 'pctSingleBid' values
#     for e in u.edges():
#         normalized_value = norm(edge_pctSingleBid[e])
#         edge_color[e] = cmap(normalized_value)

#     # Prepare a vertex property map with black color for each vertex
#     vertex_color = u.new_vertex_property('vector<double>')
#     for v in u.vertices():
#         vertex_color[v] = [0, 0, 0, 1]  # RGBA for black    

#     # Prepare the dictionary with drawing parameters
#     drawing_parameters = {
#         'edge_color': edge_color,
#         'edge_pen_width': gt.prop_to_size(edge_count, mi=0.5, ma=25, power=1),
#         'vertex_fill_color': vertex_color
#     }
    
#     return drawing_parameters

In [ ]:
##Code how to convert a graph-tool object into a networkx object
import networkx as nx

G_AT_2008 = nx.Graph()

for v in datasets['AT_2008'].vertices():
    # Here, 'v_prop' is a dictionary of the vertex properties
    v_prop = {prop: datasets['AT_2008'].vp[prop][v] for prop in datasets['AT_2008'].vertex_properties.keys()}
    G_AT_2008.add_node(v, **v_prop)

# Add edges and their attributes to the networkx graph
for e in datasets['AT_2008'].edges():
    # Here, 'e_prop' is a dictionary of the edge properties
    e_prop = {prop: datasets['AT_2008'].ep[prop][e] for prop in datasets['AT_2008'].edge_properties.keys()}
    G_AT_2008.add_edge(e.source(), e.target(), **e_prop)

In [ ]:
# How to plot a networkx graph using all the properties
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

# Assuming 'G_nx' is your networkx graph and it has '_pos' as a node attribute,
# 'count' as an edge attribute for weight, and 'pctSingleBid' for edge color

# Extract the largest connected component
largest_cc = max(nx.connected_components(datasets_nx['AT_2008']), key=len)
G_largest = datasets_nx['AT_2008'].subgraph(largest_cc)

# Extract the position
pos = {node: data['_pos'] for node, data in G_largest.nodes(data=True)}

# Get weights and colors from the edges
edge_colors = [data['pctSingleBid'] for _, _, data in G_largest.edges(data=True)]
weights = [data['count'] for _, _, data in G_largest.edges(data=True)]

# Normalize edge weights
max_weight = max(weights)
edge_widths = [w / max_weight * 3 for w in weights]  # Scale weights for visibility

# Map 'pctSingleBid' to a color gradient
# Here, 0 is blue and 1 is red
edge_color_map = [plt.cm.Reds(pct) for pct in edge_colors]

# Draw the largest connected component of the graph
plt.figure(figsize=(12, 12))
nx.draw_networkx_edges(G_largest, pos, edge_color=edge_color_map, width=edge_widths)
nx.draw_networkx_nodes(G_largest, pos, node_size=50, node_color='lightblue')

# Remove labels for nodes
plt.title('Largest Connected Component with Custom Attributes')
plt.axis('off')  # Turn off the axis
plt.show()


In [ ]:
##Code to draw projections in networx
import matplotlib.pyplot as plt
import networkx as nx

# Function to draw the largest connected component of a bipartite projection graph
def draw_largest_component_projection(G, title):
    # Extract the largest connected component
    if nx.is_connected(G):
        G_largest = G
    else:
        largest_cc = max(nx.connected_components(G), key=len)
        G_largest = G.subgraph(largest_cc).copy()

    # Use the spring layout for the largest component
    pos = nx.spring_layout(G_largest, seed=42)  # for consistent layout
    plt.figure(figsize=(8, 6))
    nx.draw(G_largest, pos, with_labels=True, node_color='lightblue', edge_color='gray', node_size=2500, font_size=16)
    plt.title(title)
    plt.show()

# Draw the largest connected component of the issuer projection
draw_largest_component_projection(issuer_projection, 'Largest Component - Issuer Projection')

# Draw the largest connected component of the winner projection
draw_largest_component_projection(winner_projection, 'Largest Component - Winner Projection')


In [ ]:
# How to plot a spring layout graph in networkx
import matplotlib.pyplot as plt
import networkx as nx

# Assuming 'G_nx' is your networkx graph

# Find the largest connected component
largest_cc = max(nx.connected_components(datasets_nx['AT_2008']), key=len)

# Create a subgraph of G_nx consisting only of this component
G_largest = datasets_nx['AT_2008'].subgraph(largest_cc).copy()

# Use the spring layout for the largest component
pos = nx.spring_layout(G_largest, seed=42)  # Seed for reproducibility

# Draw the largest connected component of the graph
plt.figure(figsize=(12, 12))  # Set the size of the plot
nx.draw(G_largest, pos, with_labels=True, node_size=700, node_color='lightblue', edge_color='gray')
plt.title('Largest Connected Component with Spring Layout')
plt.show()



In [ ]:
##This one worked, but partioned the data according to the best bipartition (mathematically speaking)
# Which is of course not what we want
import graph_tool.all as gt
from itertools import combinations

# Assuming 'g' is your bipartite graph obtained from datasets['AT_2008']
g = datasets['AT_2008']
u = gt.extract_largest_component(g, prune=True)

# Check if the graph is bipartite and get the partition if it is
is_bipartite, partition_map = gt.is_bipartite(u, partition=True)

# We need to work on a copy to avoid changing the original graph
g_copy = gt.Graph(u)

# Project the bipartite graph onto one of its vertex sets.
# Let's say we want to project onto the set where partition_map == 0
for v in g_copy.vertices():
    if partition_map[v] == 0:
        # Get all neighbors of v that are in the other set (partition_map == 1)
        neighbors = [u for u in v.out_neighbours() if partition_map[u] == 1]
        # Add edges between all neighbors in the projection
        for u, w in combinations(neighbors, 2):
            g_copy.add_edge(u, w)

# Create a new graph from the copy, filtered to keep only the vertices from one set
projected_graph = gt.GraphView(g_copy, vfilt=lambda v: partition_map[v] == 1)
projected_graph = gt.Graph(projected_graph, prune=True)  # To create a standalone graph

# Draw the projected graph
pos = gt.sfdp_layout(projected_graph)
gt.graph_draw(projected_graph, pos=pos, vertex_text=projected_graph.vertex_index, output_size=(1000, 1000))


In [ ]:
#Failed code to project the bipartite graph onto one of its vertex sets in graph tool manually
import graph_tool.all as gt
from itertools import combinations

# Assuming 'g' is your bipartite graph obtained from datasets['AT_2008']
g = datasets['AT_2008']
u = gt.extract_largest_component(g, prune=True)

# Assuming that vertex labels are given as a property map
labels = u.vertex_properties["label"]

# Initialize two property maps to distinguish issuers from winners
is_issuer = u.new_vertex_property('bool')
is_winner = u.new_vertex_property('bool')

# Assign vertices to issuers or winners based on the label suffix
for v in u.vertices():
    if labels[v].endswith("_i"):
        is_issuer[v] = True
        is_winner[v] = False
    elif labels[v].endswith("_w"):
        is_issuer[v] = False
        is_winner[v] = True

# Now we create projections based on these property maps
# Let's start with the issuer projection
issuer_projection = gt.GraphView(u, vfilt=is_issuer)
issuer_projection = gt.Graph(issuer_projection, prune=True)  # To create a standalone graph

# Now we add edges between issuers if they share a common winner
for v in issuer_projection.vertices():
    # Get all winners connected to the issuer
    connected_winners = [w for w in v.out_neighbours() if is_winner[w]]
    # Add edges between all issuers connected to the same winner
    for w1, w2 in combinations(connected_winners, 2):
        issuer_projection.add_edge(issuer_projection.vertex(w1), issuer_projection.vertex(w2))

# The winner projection would be similar, just invert the roles of issuers and winners
winner_projection = gt.GraphView(u, vfilt=is_winner)
winner_projection = gt.Graph(winner_projection, prune=True)  # To create a standalone graph

for v in winner_projection.vertices():
    # Get all winners connected to the issuer
    connected_issuer = [i for i in v.out_neighbours() if is_issuer[i]]
    # Add edges between all issuers connected to the same winner
    for i1, i2 in combinations(connected_issuer, 2):
        winner_projection.add_edge(winner_projection.vertex(i1), winner_projection.vertex(i2))

# Draw the issuer projection graph
pos = gt.sfdp_layout(issuer_projection)
gt.graph_draw(issuer_projection, pos=pos, vertex_text=issuer_projection.vertex_properties['label'], output_size=(1000, 1000))

# Draw the issuer projection graph
pos = gt.sfdp_layout(winner_projection)
gt.graph_draw(winner_projection, pos=pos, vertex_text=winner_projection.vertex_properties['label'], output_size=(1000, 1000))


In [ ]:
# This worked!

# Parse all nodes and extract their attributes to identify issuers and winners
issuer_nodes = set()
winner_nodes = set()

for node in root.findall("{http://graphml.graphdrawing.org/xmlns}graph/{http://graphml.graphdrawing.org/xmlns}node"):
    node_id = node.get('id')
    for data in node.findall("{http://graphml.graphdrawing.org/xmlns}data"):
        # We assume that the key for the issuer/winner label is consistent across nodes as "key4"
        if data.get('key') == 'key4':
            if data.text.endswith('_i'):
                issuer_nodes.add(node_id)
            elif data.text.endswith('_w'):
                winner_nodes.add(node_id)

# Now let's create the bipartite network correctly and project it for issuers
B = nx.Graph()
# Add nodes with their respective sets
B.add_nodes_from(issuer_nodes, bipartite=0)
B.add_nodes_from(winner_nodes, bipartite=1)

# Add edges between issuers and winners
for edge in root.findall("{http://graphml.graphdrawing.org/xmlns}graph/{http://graphml.graphdrawing.org/xmlns}edge"):
    source = edge.get('source')
    target = edge.get('target')
    if source in issuer_nodes or target in issuer_nodes:
        B.add_edge(source, target)

# Project the bipartite network for issuers
issuer_projection = nx.bipartite.projected_graph(B, issuer_nodes)

# Now visualize the projection again
pos_issuer_projection = nx.spring_layout(issuer_projection)

plt.figure(figsize=(12, 12))
nx.draw(issuer_projection, pos_issuer_projection, with_labels=False, node_size=50, font_size=10)
plt.title('Issuer Projection of the Network')

# Save the graph to a file
issuer_projection_path = '/mnt/data/graph_issuer_projection_fixed.png'
plt.savefig(issuer_projection_path)

plt.show()

issuer_projection_path, len(issuer_projection.nodes()), len(issuer_projection.edges())


In [ ]:
## Draw the new projection using graph tool

u = gt.extract_largest_component(datasets_prj['AT_2016_issuer'], prune=True)
pos = gt.sfdp_layout(u)
gt.graph_draw(u, pos=pos, output_size=(1000, 1000), output="issuer_projection.png")


In [ ]:
# Ensure the output directory exists for the stored positions
output_dir_nbm2 = "nested_blockmodel_images2"
os.makedirs(output_dir_nbm2, exist_ok=True)

In [ ]:
# Process each graph to create a nested blockmodel and visualize it
for dataset_key, g in datasets_prj.items():
    u = gt.extract_largest_component(g, prune=True)

    # Minimize the nested blockmodel
    state = gt.minimize_nested_blockmodel_dl(u)

    # Get the hierarchy tree and compute the layout
    t = gt.get_hierarchy_tree(state)[0]
    tpos = gt.radial_tree_layout(t, t.vertex(t.num_vertices() - 1, use_index=False), weighted=True)
    cts = gt.get_hierarchy_control_points(u, t, tpos)
    pos = u.own_property(tpos)

    # Get the block labels for coloring the vertices
    b = state.levels[0].b

    # Copy the block labels and take modulo for shaping
    shape = b.copy()
    shape.a %= 14  # This will result in 14 different shapes

    # Draw the graph with the computed layout and save it
    output_path = os.path.join(output_dir_nbm2, f"{dataset_key}.png")
    gt.graph_draw(u, pos=pos, vertex_fill_color=b, vertex_shape=shape,
                  edge_control_points=cts, edge_color=[0, 0, 0, 0.3],
                  vertex_anchor=0, output=output_path)

In [ ]:
# Get the list of all saved images for the stored positions
image_files_nbm2 = sorted([os.path.join(output_dir_nbm2, f) for f in os.listdir(output_dir_nbm2) if f.endswith('.png')])

# Calculate the number of rows and columns for the facet wrap
cols = 4
rows = int(np.ceil(len(image_files_nbm2) / float(cols)))

# Create the figure with subplots for the stored positions
fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 5))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Loop through each image file for the stored positions and add it to the subplot
for i, image_file in enumerate(image_files_nbm2):
    # Read each graph from its saved file
    img = plt.imread(image_file)
    axes[i].imshow(img)
    axes[i].axis('off')  # Hide the axes
    # Set the title for the subplot
    axes[i].set_title(image_file.split('/')[-1].replace('.png', ''), fontsize=10)

# Hide any unused subplots
for j in range(i + 1, len(axes)):
    axes[j].axis('off')

# Adjust the layout and show the visualization
plt.tight_layout()
plt.show()
